In [ ]:
#main.py
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision import transforms as T
from datasets.image_dataset import ImageDataset
from torch.utils.data import DataLoader
from models.customNN import FirstAttemptFCN, Model
import  torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from uuid import uuid4
import os
import torch.nn as nn
from datetime import datetime
import numpy as np
#prepare Dataset



#Modelling

#Training

if __name__ == "__main__":
    SEED= 42
    torch.manual_seed(SEED)

    #create folder
    # folder_name= f"run-{str(uuid4())[:8]}" #unique num
    #get the current date and time
    dt=datetime.now()
    #fromat the d/t with custom separators
    f_dt= dt.strftime("%Y-%m-%d-%H-%M-%S")
    folder_name= f"run-{f_dt}"
    os.mkdir(f"artifacts/{folder_name}")

    #create a tensorboard writer
    writer= SummaryWriter(log_dir=f"artifacts/{folder_name}/tensorflow_logs")


    train_csv_path=r'data\train.csv'
    val_csv_path=r'data\test.csv'
    BATCH_SIZE= 4
    transfroms= T.Compose(
        [
            T.Resize((256,256)),
            T.ToTensor()
        ]
    )
    train_dataset=ImageDataset(train_csv_path, transforms=transfroms)
    train_data_loader= DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    val_dataset=ImageDataset(val_csv_path, transforms=transfroms)
    val_data_loader= DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )


    model= FirstAttemptFCN(img_size=256, num_channels=3,num_labels=5)

    #train
    LR = 0.001
    EPOCHS= 15
    MIN_EPOCH=5
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    epochwise_train_losses=[]
    epochwise_val_losses=[]
    epochwise_val_acc=[]
    epochwise_train_acc=[]
    best_val=0


    for epoch in range(EPOCHS):
        train_running_loss=0
        val_running_loss=0
        train_running_acc=0
        val_running_acc=0


        #training
        model.train()
        for images , labels in train_data_loader:
            optimizer.zero_grad()

            model_out=model(images)
            model_out= F.log_softmax(model_out, dim=1)

            loss= criterion(model_out,labels)

            train_running_loss += loss.item() #to sum up the loss generated in each batch
            loss.backward()  #
            optimizer.step()

        # validation
        model.eval()    # change into evaluation mode
        for images , labels in train_data_loader:
            # print(images.shape, labels.shape)

            model_out=model(images)
            model_out= F.log_softmax(model_out, dim=1)

            #for accuracy
            preds = torch.argmax(model_out,dim=1)
            acc = (preds==labels).float().mean()  #running accuracy
            train_running_acc += acc.item()


        for images , labels in val_data_loader:
            # print(images.shape, labels.shape)

            model_out=model(images)
            model_out= F.log_softmax(model_out, dim=1)

            #for accuracy
            preds = torch.argmax(model_out,dim=1)
            acc = (preds==labels).float().mean()  #running accuracy
            val_running_acc += acc.item()

            loss= criterion(model_out,labels)
            val_running_loss += loss.item() #to sum up the loss

        #losses
        avg_train_loss= train_running_loss/len(train_data_loader) #total loss minus no of batch
        avg_val_loss= val_running_loss/len(val_data_loader) #total loss minus no of batch


        epochwise_train_losses.append(avg_train_loss)
        epochwise_val_losses.append(avg_val_loss)


        #accuracy
        avg_train_acc= train_running_acc/len(train_data_loader)
        avg_val_acc= val_running_acc/len(val_data_loader)

        epochwise_val_acc.append(avg_val_acc)


        #log to tensorboard

        writer.add_scalar("Loss/Train",avg_train_loss,epoch)
        writer.add_scalar("loss/Val",avg_val_loss,epoch)
        writer.add_scalar("Accuracy/Train",avg_train_acc,epoch)
        writer.add_scalar("Accuracy/Val",avg_val_acc,epoch)
        #early stopping criteria
        if (epoch >= MIN_EPOCH):
            last_5_val_avg = np.mean(epochwise_val_acc[-5:])
            first_5_val_avg = np.mean(epochwise_val_acc[-6:-1])
            diff= last_5_val_avg - first_5_val_avg
            if diff < 0.001:
                break

        print(f"Epoch{epoch} Train Loss:{avg_train_loss:.3f}\t train_accuracy:{avg_train_acc:.2f} \t Val Loss:{avg_val_loss:.2f} \t val_accuracy:{avg_val_acc:.2f}")


        checkpoint_name=f"artifacts/{folder_name}/ckpt--{model.__class__.__name__}--val-acc-{avg_val_acc:.2f}--epoch-{epoch}.pth"
        checkpoint={
            "epoch":epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_loss":avg_train_loss,
            "val_loss":avg_val_loss,
            "train_acc":avg_train_acc,
            "val_acc":avg_val_acc
        }
        #save the model
        # torch.save(checkpoint,checkpoint_name)
        # if avg_val_acc>best_val:
        #     best_val=avg_val_acc
        #     torch.save(model.state_dict(),f"artifacts/{folder_name}/--best-model-val-acc-.pth")






    #plot the losses and accuracy
    # fig,(ax1,ax2)= plt.subplots(1,2,figsize=(16,9))
    # x_axis_values = np.arange(0,epoch + 1,1)

    # ax1.plot(x_axis_values,epochwise_train_losses,label="train_loss")
    # ax1.plot(x_axis_values,epochwise_val_losses,label="val_loss")
    # ax1.set_title("Train Losses v/s validation Losses")
    # ax1.legend()

    # ax2.plot(x_axis_values,epochwise_train_acc,label="train_acc")
    # ax2.plot(x_axis_values,epochwise_val_acc,label="val_acc")
    # ax2.set_title("Train accuracy v/s validation accuracy")
    # ax2.legend()

    # plt.show()

    #load model

    # model.load_state_dict()

In [ ]:
#train.py

import numpy as np
import os
import pickle
from os.path import join
from sklearn.neighbors import KNeighborsClassifier
from utils.io import read_as_csv
from utils.preprocessing import read_image, image_transforms,label_transform
from utils.preprocessing import label_to_index
from config import MODEL_CHECKPOINT_PATH, ROOT_DIR

##these are the datapoints grouped.
#load csv
train_files, train_labels= read_as_csv("data/train.csv")

if __name__ == "__main__":
    x_train= [ image_transforms(filename,label,root_dir=ROOT_DIR) for filename,label in zip(train_files,train_labels)]

    y_train=[label_transform(label) for label in train_labels ]

    clf= KNeighborsClassifier(n_neighbors=4)
    clf.fit(x_train,y_train)
    print("The train score is:",clf.score(x_train,y_train))

    # save model
    with open (MODEL_CHECKPOINT_PATH,'wb') as file:
        pickle.dump(clf,file)






In [ ]:
#evaluate.py

#import checkpoint

from config import MODEL_CHECKPOINT_PATH, ROOT_DIR
from utils.io import read_as_csv, save_as_csv,read_predictions
import pickle
import yaml
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from utils.preprocessing import label_to_index, image_transforms,label_transform,label_index,index_label
from viz.display_image_grid import display_grid
from utils.load_config import config_loader
from app import predict
from PIL import Image
import pandas as pd


def get_predictions(model_checkpoint,test_file, out_dir):

    #load the model using
    with open (model_checkpoint,'rb') as file:
        model=pickle.load(file)

    test_filename, test_labels= read_as_csv(test_file)

    LABELS = label_index.keys()
    #predict on test data and save it somewhere
    x_test= [ image_transforms(filename,label,root_dir = "data/genetic-syndrome-dataset") for filename,label in zip(test_filename, test_labels)]
    y_test=[label_transform(label) for label in test_labels ]
    y_pred= model.predict(x_test)
    y_pred_labels=[ index_label[y_pred_index] for y_pred_index in y_pred ]
    # print(y_pred_labels)
    save_as_csv(test_filename, test_labels, out_dir,y_pred_labels)


##
def evaluate(out_directory):
    file_name,label,pred_label= read_predictions(out_directory)
    y_test=[label_transform(label) for label in label ]
    y_pred= [label_transform(pred_label) for y_pred_index in pred_label ]
    print(y_test,y_pred)



if __name__== "__main__":
    # outdir= 'results/predictions.csv'
    configs= config_loader("configs\evaluate_configs.yaml")
    # get_predictions(configs.get("checkpoint_path"),configs.get("test_csv"),configs.get("out_dir"))
    # read_predictions(configs.get("out_dir"))
    evaluate(configs.get("out_dir"))

    # y_pred= model.predict(x_test)
    # cm = confusion_matrix(y_test, y_pred)
    # disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=LABELS)
    #
    # print(LABELS) neede=s to be uncomment
    # print(cm)
    # disp.plot()

    # plt.show()
    # print("The test score is:", model.score(x_test,y_test))

    # print("y predict",y_pred)
    # print(classification_report(y_true=y_test, y_pred=y_pred, target_names=LABELS))

    # print(display_grid(ROOT_DIR,test_filename, test_labels,3,4,'Genetic Syndrome',y_pred_labels))  till here

    # target_names=['']







    #score print score



In [ ]:
#app.py

import torch
import requests
from PIL import Image
from torchvision import transforms
import gradio as gr
import torch.nn as nn
import  torch.nn.functional
import torchvision.transforms as T
from models.customNN import FirstAttemptFCN, Model
from utils.io import read_as_csv


model_path= r"artifacts\run-2023-11-29-16-14-49\ckpt--FirstAttemptFCN--val-acc-0.72--epoch-1.pth"
checkpoint= torch.load(model_path)
model_state_dict = checkpoint['model_state_dict']
model= FirstAttemptFCN(img_size=256, num_channels=3,num_labels=5)
# model.load_state_dict(model_state_dict)
model.eval()
# print(checkpoint)
transform= T.Compose([T.Resize(size = (256,256)), T.ToTensor()])

labels=["Angleman", "apert", "charge", "down", "williams","Cannot identify"]
def predict(inp):
  inp = transform(inp).unsqueeze(0)
  with torch.no_grad():

    prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)

    confidences = {labels[i]: float(prediction[i]) for i in range(prediction.shape[0])}
    # max= list(sorted(confidences.values()))[-1]
    # second_max= list(sorted(confidences.values()))[-2]
    # if(max-second_max>0.9):
    #   return(confidences)
    # else:
    #   return('cannot identify')


  # return confidences, prediction
  return confidences

#











gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=5),
             examples=[r"data\genetic-syndrome-dataset\AngleMan\ndl4wvs.jpg",
                       r"data\genetic-syndrome-dataset\charge\charge 5.jpg",

             ]
).launch()

#

In [ ]:
#preprocessing.py


from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
from utils.io import get_image_label_pairs
from utils.io import list_files
import os
from os.path import join


def image_transforms(filename,label,root_dir):
    ##since the image is inside the folder so we first join dir filename
    ##then we finally join all to get the required path
    """ Extracts the image from the file name and label
        filename:string
        label:string
        root_dir: string """
    file_path = os.path.join(root_dir,label, filename)

    ##we perform required transforms

    img_arr=read_image(file_path, mode="zoom",size=(64,64),grayscale=True)
    flattened_img_arr= img_arr.flatten()
    # print(flattened_img_arr.shape)
    return flattened_img_arr

def label_transform(label):
    index= label_to_index(label)
    return index
def read_image(image_path:str,mode:str, size=(100,100), grayscale:bool=False) -> np.ndarray:
    """
    TODO: resize to a default value
    args:
    -------
    image_path: str filepath of the image we want to read.
    mode: str should be either zoom or padding, this argument basically defines either zoom or padding at the border.
    size: tuple that consists width and height
    """

    with Image.open(image_path) as im:
        width,height= im.size
        changed_image = im
        if width!=height:
            #if the size is not same, we are trying to make the size same , either by zoom or padding .
            #  After the size is equl , resizing is done.
            if mode=='zoom':
                 if(width>height):
                    #we need to cut from the width
                    crop_size=(width-height)/2
                    left, top, right, bottom = crop_size,0,width-crop_size,height
                    changed_image=im.crop((left, top, right, bottom))
                    # print('after cropping',changed_image.size)
                 else:
                    #we need to cut from the height
                    crop_size=(height-width)/2
                    left, top, right, bottom =  0, crop_size  ,width, height-crop_size
                    changed_image=im.crop((left, top, right, bottom))
                    # print('after cropping',cropped_image.size)

            elif mode=='padding':
                   changed_image= ImageOps.pad(im,size)
                #    print()


        resized_image = changed_image.resize(size,resample=0).convert("RGB")
        img1_array=np.asarray(resized_image)
        # print(img1_array)
        return(img1_array)




# def display_grid(image_dir,images,labels: list,n_rows:int,n_cols:int,title:str):
#     fig,axes = plt.subplots(n_rows,n_cols)
#     fig.suptitle(title)
#     idx=0
#     for i in range(n_rows):
#             for j in range(n_cols):

#                 image_list =join(IMAGE_DIR,image_file_name[idx])

#                 img_arr = read_image(image_list,mode='padding',size=(100,100),grayscale=True)
#                 axes[i][j].imshow(img_arr)
#                 axes[i][j].set_title(f"label: {labels[idx]}")
#                 axes[i][j].set_xticks([])
#                 axes[i][j].set_yticks([])
#                 idx+= 1

#     plt.show()


label_index = { 'AngleMan':0, 'Apert': 1,'charge':2, 'Down':3,'Williams':4}
index_label = {index:label for label, index in label_index.items()}


def label_to_index(label):
    try:
        return label_index[label]
    except KeyError:
        raise KeyError(f"Label:{label} not found try one of these: {list(label_index.keys())}")

def index_to_label(index):
     try:
        return(index_label[index])
     except KeyError:
         raise KeyError(f"Index not found try one of these: {list(index_label.keys())}")







if __name__ == "__main__":
    # print("hello from preprocessing")

    # IMAGE_PATH = r'data\genetic-syndrome-dataset\Apert\njug98f.jpg'
    # image = read_image(IMAGE_PATH)

    # plt.imshow(image)
    # #removeticks
    # plt.xticks([])
    # plt.yticks([])


    # #add title#plot image
    # plt.title("Apert Syndrome")

    from viz.display_image_grid import display_grid
    IMAGE_DIR="data\genetic-syndrome-dataset\Apert"
    imggg_path,labels = get_image_label_pairs(IMAGE_DIR, 'Apert')


    image_file_name= list_files(IMAGE_DIR,'')
    # ###to print label to index
    # print(label_to_index("Apert"))
    # # ##to print index to label
    # print(index_to_label(66))


    display_grid(IMAGE_DIR,image_file_name,labels,3,4,'Genetic Syndrome')


    # sample_img_path =r'data\genetic-syndrome-dataset\Apert\pl8tkd4.jpg'
    # sample_img_path_2 =r'data\genetic-syndrome-dataset\Apert\qh5g95a.jpg'

    # img = Image.open(sample_img_path_2)
    # print('original image size',img.size)
    # img = read_image(image_path=sample_img_path_2,mode='zoom')
    # print(img.shape)
    # # plt.imshow(img)
    # plt.show()
    ##dictionary for label and index


In [ ]:
#visualizing

from config import MODEL_CHECKPOINT_PATH
import pickle
from utils.io import read_as_csv
from utils.preprocessing import image_transforms,label_transform,read_image
from utils.io import read_as_csv
from config import MODEL_CHECKPOINT_PATH, ROOT_DIR
import matplotlib.pyplot as plt


with open (MODEL_CHECKPOINT_PATH,'rb') as file:
    model=pickle.load(file)
test_filename, test_labels= read_as_csv("data/test.csv")
x_test= [ image_transforms(filename,label,root_dir = "data/genetic-syndrome-dataset")
         for filename,label in zip(test_filename, test_labels)]

train_files, train_labels= read_as_csv("data/train.csv")
x_train= [ image_transforms(filename,label,root_dir=ROOT_DIR)
           for filename,label in zip(train_files,train_labels)]

y_train=[label_transform(label) for label in train_labels ]

graph=model.kneighbors(x_test[:4],n_neighbors=5,return_distance=False)
print(graph)
print(type(graph))

fig, axes = plt.subplots(4,6 ,figsize=(10, 10))

for i in range(4):
    axes[i, 0].imshow(x_test[i].reshape(64, 64,3))
    axes[i, 0].set_title(f"Test Image {i+1}")
    axes[i, 0].axis('off')

    for j, idx in enumerate(graph[i]):
        axes[i, j+1].imshow(x_train[idx].reshape(64, 64,3))
        axes[i, j+1].axis('off')
        axes[i, j+1].set_title(f"nearest neighbors {j}")


#     file_path = os.path.join(root_dir,label, filename)
#     img_arr=read_image(file_path, mode="zoom",size=(64,64),grayscale=True)
#     flattened_img_arr= img_arr.flatten()
#     return flattened_img_arr

plt.tight_layout()
plt.show()




In [ ]:
#requirement.txt
numpy
scikit-learn==1.2.1
matplotlib==3.7.2
torch
torchvision

In [ ]:
#io.py
import os
import csv


def list_files(dir: str, file_extension: str) -> list:
    if not os.path.isdir(dir):
        return None
    files = os.listdir(dir)
    return files


def get_image_label_pairs(images_path: str, label: str):
    image_file_names = list_files(images_path, "jpg")
    label_name = [label] * len(image_file_names)
    return (image_file_names, label_name)

# def read_as_csv(image_paths, labels, outfile):
#     with open(outfile,"r")

def save_as_csv(image_paths, labels, outfile,pred_lables=None):
    with open(outfile, "w", newline="") as f:
        writer = csv.writer(f)
        if pred_lables:
            writer.writerow(["file", "label","predicted_labels"])

            for image_path, label, pred_lables in zip(image_paths, labels,pred_lables):
                writer.writerow([image_path, label,pred_lables])
        else:
            writer.writerow(["file", "label","predicted_labels"])

            for image_path, label in zip(image_paths, labels,pred_lables):
                writer.writerow([image_path, label])
def read_as_csv(csv_path):
    file_name_arr=[]
    label_arr=[]
    with open(csv_path,'r') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
             file_name_arr.append(row[0])
             label_arr.append(row[1])
        return(file_name_arr,label_arr)

def read_predictions(csv_path):
    file_name_arr=[]
    label_arr=[]
    pred_label_arr=[]
    with open(csv_path,'r') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
             file_name_arr.append(row[0])
             label_arr.append(row[1])
             pred_label_arr.append(row[2])
        return(file_name_arr,label_arr,pred_label_arr)







if __name__ == "__main__":
    # z=list_files('data\genetic-syndrome-dataset\AngleMan','jpg')
    # print(z)
    # images_1,labels_1 =get_image_label_pairs('data\genetic-syndrome-dataset\AngleMan','Angleman')
    # images_2,labels_2 =get_image_label_pairs('data\genetic-syndrome-dataset\Apert','Apert')

    # save_as_csv(images_1,labels_1,'data/Angleman.csv')
    # save_as_csv(images_2,labels_2,'data/Apert.csv')

    # img_folders = ["Angleman", "apert", "charge", "down", "williams"]
    # x = []
    # y = []
    # for image in img_folders:
    #     image_path = f"data\genetic-syndrome-dataset\{image}"
    #     images, labels = get_image_label_pairs(image_path, image)
    #     x.extend(images)
    #     y.extend(labels)

    # # save_as_csv= f'{x},{y},'data/.csv'
    # save_as_csv(x, y, "data/genetics_entire.csv")


    ##gives us diffrent column of file name and label
   file_name, label=read_as_csv("'data/test.csv'","data/train.csv")
#    print(label)



In [ ]:
#prepare dataset
import glob
from utils.io import get_image_label_pairs
from utils.io import save_as_csv

BASE_DIR= 'data\Ear Domain\Datos'
TEST_DIR= ' data\Ear Domain\Datos\Testing'
TRAIN_DIR = 'data\Ear Domain\Datos\Training-validation'
img_folder_name= ['Chronic otitis media','Earwax plug','Myringosclerosis','Normal']
train_images=glob.glob(f"{TRAIN_DIR}/**/*.jpg")
test_images=glob.glob(f"{TEST_DIR}/**/*.jpg")

# print(train_label)
train_label=[label.split("//")[:4] for label in train_images]
test_label=[label.split("//")[:4] for label in test_images]

save_as_csv(train_images, train_label, "data/Ear Domain/Datos/train.csv")
save_as_csv(test_images, test_label, "data/Ear Domain/Datos/test.csv")
# x = []
# y = []
# for image in img_folder_name:
#         image_path = f"data\Ear Domain\Datos\Testing{image}"
#         images, labels = get_image_label_pairs(image_path, image)
#         x.extend(images)
#         y.extend(labels)

# save_as_csv(x, y, "data/Ear Domain/Datos/test.csv")
# for image in img_folder_name:
#         image_path = f"data\Ear Domain\Datos\Training-validation{image}"
#         images, labels = get_image_label_pairs(image_path, image)
#         x.extend(images)
#         y.extend(labels)

    # save_as_csv= f'{x},{y},'data/.csv'









In [ ]:
#traintestsplit

import os
import argparse

from os.path import join
from utils.io import get_image_label_pairs
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from utils.io import save_as_csv
# DATA_DIR = 'data\genetic-syndrome-dataset'
#using argparse

#creating an instance of class Argumentparser
parser= argparse.ArgumentParser(prog='')
parser.add_argument("-d",'--data_dir',type=str, help='Consist path of the images in directory')
# parser.add_argument('--label',action= 'append', help='list of the name of labels from the folder')
parser.add_argument('--test_size',type=float, help='test size of the train and test')
parser.add_argument('--random_state',type=int, help='random_slate of the train and test')

args= parser.parse_args()

print(args.data_dir)
DATA_DIR= args.data_dir

data_folders = os.listdir(DATA_DIR)


X=[]
Y=[]

for folder in data_folders:
    data_path = join(DATA_DIR,folder)
    label=folder

    files_name,labels= get_image_label_pairs(data_path, label)
    X.extend(files_name)
    Y.extend(labels)

print(len(X),len(Y))

#split the data into train and test sets

X_test, X_train, Y_test, Y_train = train_test_split(X,Y,test_size=0.20,random_state=42)
print("X test size:", len(X_test))
print("Y test size:", len(Y_test))





# save_as_csv(X_test,Y_test,'data/test.csv')
# save_as_csv(X_train,Y_train,'data/train.csv')




In [ ]:
#image dataset
from utils.preprocessing import read_image, image_transforms,label_transform
from utils.preprocessing import label_to_index
from utils.io import read_as_csv
from config import MODEL_CHECKPOINT_PATH, ROOT_DIR
from torch.utils.data import Dataset, DataLoader
from os.path import join
import matplotlib.pyplot as plt
from PIL import Image




class GeneticDataset(Dataset):
    def __init__(self,csv_file):
        train_files, train_labels= read_as_csv(csv_file)
        self.imgs= [ image_transforms(filename,label,root_dir=ROOT_DIR) for filename,label in zip(train_files,train_labels)]
        self.labels=[label_transform(label) for label in train_labels ]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index=0):
        # return self.imgs[index],self.labels[index]

        pass



class ImageDataset(Dataset):
    def __init__(self,csv_path,transforms:None):
        images,labels=read_as_csv(csv_path)
        self.images = images
        self.labels=labels
        self.transforms=transforms

    def __len__(self):
        return len(self.images)

    def __str__(self):
        return f"<ImageDataset with {self.__len__()} samples>"
    def __getitem__(self,index):
        # image=self.images[index]
        # label=self.labels[index]
        image_name=self.images[index]
        label_name= self.labels[index]

        image_path= join("data","genetic-syndrome-dataset",label_name,image_name)
        image= Image.open(image_path).convert('RGB')
        label=label_to_index(label_name)
        if self.transforms:
            image= self.transforms(image)
        return image,label



In [ ]:
#configs.yaml
data_root: data
train_csv: train1.csv
test_csv: test1.csv
models:
  name: kneighbors
  args:
      n_neighbors: 3
checkpoint_path: models/finalized_knn_model.sav

In [ ]:
#load config
import yaml
from argparse import ArgumentParser


def config_loader(config_file):
    config={}
    with open(config_file,"r")as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    return config


In [ ]:
#evaluateconfigs

data_root: data
train_csv: train1.csv
test_csv: test1.csv
out_dir: results/predictions.csv
checkpoint_path: models/knn_model.pkl
test_csv: 'data/test.csv'


In [ ]:
#model
import torch
from torch.nn.functional import relu
from torch.nn import Linear
import torch.nn as nn

class FirstAttemptFCN(torch.nn.Module):
    """
    subclassing torch.nn.module gives
      us automatic gradient calculation

    Args:
        torch (_type_): _description_
    """
    def __init__(self, img_size:int, num_channels,num_labels) -> None:
        super().__init__()
        self.fc1=Linear(img_size*img_size*num_channels,128)
        self.fc2=Linear(128, num_labels)

    def forward(self,x: torch.tensor):
        x=x.reshape(x.shape[0], -1)
        return self.fc2(relu(self.fc1(x)))

class SecondAttemptFCN(torch.nn.Module):

    def __init__(self,img_size:int,num_labels):
        super().__init__()
        # self.fc21= Linear(img_size*img_size,512)
        # self.fc22= Linear(512,256)
        # self.fc23 =Linear(256,128)
        # self.fc24 =Linear(256,num_labels)
        self.nn= torch.nn.Sequential(
            Linear(img_size*img_size,512),
            torch.nn.ReLU(),
            Linear(512,128),
            torch.nn.ReLU(),
            Linear(128,num_labels)
        )

    def forward(self, x:torch.tensor):
        x=x.reshape(-1)
        return self.nn(x)


class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv = nn.Conv2d(3,3,kernel_size=3,padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.conv_1 = nn.Conv2d(3,1,kernel_size=3,padding=1)
        self.linear_1=nn.Linear(1*32*32,4)
        # self.linear_2 = nn.Linear(10,2)
        # self.softmax=nn.Softmax()


    def forward(self,x):
        x=  self.conv(x)
        x=  self.max_pool(x)
        x=  self.conv(x)
        x=  self.max_pool(x)
        x=  self.conv_1(x)
        x=  self.max_pool(x)
        x=self.linear_1(x.view(1,-1))
        return x



